# Trabajo Práctico Nro. 4
### Ing. Javier Ouret - 2025 - UCA - Facultad de Ingeniería
## Cliente Servidor utilizando RestAPI. (Entrega 31/06/2025)

#### NOTA: este TP usa muchas ventanas terminales. Instalar Tilix para que sea más cómodo el cambio entre las pantallas.

#### Cliente en C que actúa como un sensor IoT, publicando datos al broker MQTT (Mosquitto).

- Se conecta al broker MQTT.
- Publica datos periódicamente (temperatura, CO₂, presión).
- Usa protocolo MQTT 3.1/3.1.1 compatible con Flask.
- Broker MQTT en funcionamiento (Mosquitto en localhost:1883).
- Librería MQTT para C: paho.mqtt.c

sensor_mqtt.c (luego cambiar a sensor_temp.c)

In [1]:
%%writefile sensor_mqtt.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include "MQTTClient.h"

#define ADDRESS     "tcp://localhost:1883"
#define CLIENTID    "SensorTempC"
#define TOPIC       "sensors/temp"
#define QOS         1
#define TIMEOUT     10000L

int main() {
    MQTTClient client;
    MQTTClient_connectOptions conn_opts = MQTTClient_connectOptions_initializer;
    MQTTClient_create(&client, ADDRESS, CLIENTID,
                      MQTTCLIENT_PERSISTENCE_NONE, NULL);

    conn_opts.keepAliveInterval = 20;
    conn_opts.cleansession = 1;

    if (MQTTClient_connect(client, &conn_opts) != MQTTCLIENT_SUCCESS) {
        printf("Error conectando al broker MQTT\n");
        return 1;
    }

    printf("Conectado al broker MQTT\n");

    for (int i = 0; i < 100; i++) {
        char payload[100];
        float valor = 20.0 + rand() % 10 + ((float) rand() / RAND_MAX);  // valor entre 20–30
        snprintf(payload, sizeof(payload), "{\"sensor\":\"temp\",\"value\":%.2f}", valor);

        MQTTClient_message pubmsg = MQTTClient_message_initializer;
        pubmsg.payload = payload;
        pubmsg.payloadlen = (int)strlen(payload);
        pubmsg.qos = QOS;
        pubmsg.retained = 0;

        MQTTClient_deliveryToken token;
        MQTTClient_publishMessage(client, TOPIC, &pubmsg, &token);
        MQTTClient_waitForCompletion(client, token, TIMEOUT);

        printf("Publicado: %s\n", payload);
        sleep(5);  // Esperar 5 segundos
    }

    MQTTClient_disconnect(client, 10000);
    MQTTClient_destroy(&client);
    return 0;
}


Writing sensor_mqtt.c


Ejecutar

Debe verse:

Modificar el código anterior para simular sensores de presión y CO₂ cambiando:
- TOPIC = "sensors/presion"
- CLIENTID = "SensorPresionC"

#### Gateway Flask
- Se conecta al broker MQTT (localhost:1883)
- Se suscribe a tópicos sensors/#
- Cada vez que llega un dato:
  - Lo almacena en SQLite
  - Lo reenvía por WebSocket a la interfaz web
- Cada mensaje MQTT que llega (desde sensores C):
  - Se imprime en consola.
  - Se guarda en la base de datos.
  - Se envía a la interfaz en tiempo real vía WebSocket (sensor_data).  
#### ---    
- app.py: servidor Flask + WebSocket.
- mqtt_receiver.py: conecta al broker MQTT, guarda en BD y emite por WebSocket.
- models.py: define tabla SensorData y crea la base de datos SQLite.
- templates/index.html: página principal con canvas de gráficos.
- static/js/main.js: script que escucha WebSocket y dibuja gráficos Chart.js.
- static/css/style.css:	estilo de la interfaz.

In [9]:
%%writefile requirements.txt
Flask
Flask-SocketIO
eventlet
paho-mqtt
flask_sqlalchemy

Writing requirements.txt


Crear mqtt_receiver.py   
Este módulo corre en background como hilo del broker MQTT y se integra con SocketIO.

In [38]:
%%writefile flask_gateway/mqtt_receiver.py
import json
import paho.mqtt.client as mqtt
from datetime import datetime
from flask_socketio import SocketIO
from models import db, SensorData

BROKER = "localhost"
TOPIC = "sensors/#"

def start_mqtt(socketio: SocketIO, flask_app):
    def on_connect(client, userdata, flags, rc):
        print("Conectado al broker MQTT:", rc)
        client.subscribe(TOPIC)

    def on_message(client, userdata, msg):
        try:
            payload = json.loads(msg.payload.decode())
            print(f"MQTT recibido: {payload}")

            with flask_app.app_context():
                new_data = SensorData(
                    tipo=payload.get("sensor"),
                    valor=payload.get("value"),
                    timestamp=datetime.now()
                )
                db.session.add(new_data)
                db.session.commit()

                socketio.emit("sensor_data", {
                    "tipo": payload.get("sensor"),
                    "valor": payload.get("value"),
                    "timestamp": datetime.now().strftime("%H:%M:%S")
                })

            print("Emitido dato a WebSocket:", payload)
        except Exception as e:
            print("Error procesando mensaje MQTT:", e)

    def mqtt_thread():
        client = mqtt.Client()
        client.on_connect = on_connect
        client.on_message = on_message
        client.connect(BROKER, 1883, 60)
        client.loop_forever()

    socketio.start_background_task(mqtt_thread)


Overwriting flask_gateway/mqtt_receiver.py


app_prueba.py

In [39]:
%%writefile flask_gateway/app_prueba.py
from flask import Flask, render_template
from flask_socketio import SocketIO
from threading import Lock
import random
from datetime import datetime

app = Flask(__name__)
app.config['SECRET_KEY'] = 'clave'
socketio = SocketIO(app, cors_allowed_origins="*")

thread = None
thread_lock = Lock()

def sensor_data_thread():
    """Envía datos simulados cada 3 segundos a todos los clientes."""
    while True:
        socketio.sleep(3)
        data = {
            "tipo": random.choice(["temp", "co2", "presion"]),
            "valor": round(random.uniform(15.0, 35.0), 2),
            "timestamp": datetime.now().strftime("%H:%M:%S")
        }
        socketio.emit('sensor_data', data)
        print(f"Enviado: {data}")

@app.route("/")
def index():
    return render_template("index.html")

@socketio.on('connect')
def on_connect():
    global thread
    with thread_lock:
        if thread is None:
            thread = socketio.start_background_task(sensor_data_thread)
    print("Cliente conectado")

if __name__ == '__main__':
    socketio.run(app, host='0.0.0.0', port=5001, debug=True)



Overwriting flask_gateway/app_prueba.py


In [41]:
%%writefile flask_gateway/app.py
from flask import Flask, render_template
from flask_socketio import SocketIO
from models import db, init_db
from mqtt_receiver import start_mqtt
from datetime import datetime
from threading import Timer

app = Flask(__name__)
app.config['SECRET_KEY'] = 'clave'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///sensores.db'

db.init_app(app)
socketio = SocketIO(app, async_mode='eventlet', cors_allowed_origins="*")

@app.route('/')
def index():
    return render_template('index.html')

# Función para enviar un dato simulado después de 5 segundos
def emitir_dato_prueba():
    print("Enviando dato de prueba")
    socketio.emit("sensor_data", {
        "tipo": "temp",
        "valor": 26.5,
        "timestamp": datetime.now().strftime("%H:%M:%S")
    })

if __name__ == '__main__':
    with app.app_context():
        init_db()
        start_mqtt(socketio, app)

        # Enviar dato de prueba al iniciar
        Timer(5.0, emitir_dato_prueba).start()

    socketio.run(app, host='0.0.0.0', port=5002, debug=True, use_reloader=False)


Overwriting flask_gateway/app.py


Modelo SensorData en models.py

In [18]:
%%writefile flask_gateway/models.py
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

db = SQLAlchemy()

class SensorData(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    tipo = db.Column(db.String(50))
    valor = db.Column(db.String(50))
    timestamp = db.Column(db.DateTime, default=datetime.now)

def init_db():
    db.create_all()


Overwriting flask_gateway/models.py


In [33]:
%%writefile flask_gateway/templates/index.html
<!DOCTYPE html>
<html lang="es">
<head>
  <meta charset="UTF-8" />
  <title>Panel de Sensores con Logs</title>

  <!-- Usar socket.io v4 -->
  <script src="https://cdn.socket.io/4.7.2/socket.io.min.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>

  <link rel="stylesheet" href="{{ url_for('static', filename='css/style.css') }}" />
  <style>
    #log {
      background: #222;
      color: #eee;
      font-family: monospace;
      font-size: 12px;
      height: 120px;
      overflow-y: auto;
      padding: 8px;
      margin-bottom: 10px;
      border-radius: 5px;
    }
  </style>
</head>
<body>
  <div class="container">
    <h1>📊 Panel de Sensores IoT con Logs</h1>

    <label for="sensorFilter">Filtrar por sensor:</label>
    <select id="sensorFilter">
      <option value="todos">Todos</option>
      <option value="temp">Temperatura</option>
      <option value="co2">CO₂</option>
      <option value="presion">Presión</option>
    </select>

    <div id="log"></div>

    <canvas id="sensorChart" width="800" height="400"></canvas>
  </div>

  <script src="{{ url_for('static', filename='js/main.js') }}"></script>
</body>
</html>
l>


Overwriting flask_gateway/templates/index.html


In [34]:
%%writefile flask_gateway/static/js/main.js
const socket = io();

const ctx = document.getElementById("sensorChart").getContext("2d");
const logDiv = document.getElementById("log");

let currentFilter = "todos";

document.getElementById("sensorFilter").addEventListener("change", (e) => {
  currentFilter = e.target.value;
  clearChart();
  log(`Filtro cambiado a: ${currentFilter}`);
});

const chart = new Chart(ctx, {
  type: "line",
  data: {
    labels: [],
    datasets: [{
      label: "Valor de sensor",
      data: [],
      borderColor: "rgba(75, 192, 192, 1)",
      backgroundColor: "rgba(75, 192, 192, 0.2)",
      borderWidth: 2,
      tension: 0.3
    }]
  },
  options: {
    responsive: true,
    scales: {
      x: {
        title: { display: true, text: "Hora" }
      },
      y: {
        beginAtZero: false,
        title: { display: true, text: "Valor" }
      }
    }
  }
});

function clearChart() {
  chart.data.labels = [];
  chart.data.datasets[0].data = [];
  chart.update();
}

function log(msg) {
  console.log(msg);
  const p = document.createElement("p");
  p.textContent = msg;
  logDiv.appendChild(p);
  logDiv.scrollTop = logDiv.scrollHeight;
}

socket.on("connect", () => {
  log("Conectado al servidor WebSocket");
});

socket.on("disconnect", () => {
  log("Desconectado del servidor WebSocket");
});

socket.on("sensor_data", (data) => {
  log(`📡 Recibido: sensor=${data.tipo} valor=${data.valor} hora=${data.timestamp}`);

  if (currentFilter === "todos" || data.tipo === currentFilter) {
    chart.data.labels.push(data.timestamp);
    chart.data.datasets[0].data.push(parseFloat(data.valor));

    if (chart.data.labels.length > 30) {
      chart.data.labels.shift();
      chart.data.datasets[0].data.shift();
    }

    chart.update();
  }
});


Overwriting flask_gateway/static/js/main.js


In [35]:
%%writefile flask_gateway/static/css/style.css
body {
  font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif;
  background-color: #f4f6f8;
  color: #333;
  margin: 0;
  padding: 20px;
}

.container {
  max-width: 900px;
  margin: auto;
  padding: 20px;
  background: #ffffff;
  border-radius: 8px;
  box-shadow: 0 0 15px rgba(0,0,0,0.1);
}

h1 {
  text-align: center;
  color: #007bff;
  margin-bottom: 20px;
}

label {
  font-weight: bold;
}

select {
  padding: 6px;
  margin-left: 10px;
  margin-bottom: 20px;
}


Overwriting flask_gateway/static/css/style.css


### Consigna del TP 
**Ejercicios a realizar durante la clase. Incluir los códigos dentro de este mismo Notebook**

- Ejecutar para simular CO2 y Presión.
- Graficar
- Verificar como funciona websocket y explicar.
- Verificar como funciona MQTT y explicar.
- Explicar simulador de sensor en C.
- Mostrar resultados.